In [4]:
import numpy as np
import tensorflow as tf

from sklearn import preprocessing

## **Datos**

In [6]:
raw_csv_data = np.loadtxt("Audiobooks_data.csv", delimiter=",")

unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:, -1]

**Balanceo del Dataset**

In [12]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = list()

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

**Estandarizando las inputs**

In [13]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

**Shuffle data**

In [14]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

**Dividiendo el conjunto de datos**

In [24]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

In [25]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

## **Machine Learning**

**Datos npz**

In [26]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz["inputs"].astype(np.float)
train_targets = npz["targets"].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz["inputs"].astype(np.float), npz["targets"].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz["inputs"].astype(np.float), npz["targets"].astype(np.int)

c:\Users\Juan Pablo\.conda\envs\p37env\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\Juan Pablo\.conda\envs\p37env\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated

**Modelo**

In [39]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([    
    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
    tf.keras.layers.Dense(output_size, activation="softmax")
])

In [40]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [42]:
batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(
    train_inputs,
    train_targets,
    batch_size=batch_size,
    epochs=max_epochs,
    callbacks=[early_stopping],
    validation_data=(validation_inputs, validation_targets),
    verbose=2
)

Epoch 1/100
36/36 - 0s - loss: 0.3232 - accuracy: 0.8821 - val_loss: 0.2875 - val_accuracy: 0.8926 - 80ms/epoch - 2ms/step
Epoch 2/100
36/36 - 0s - loss: 0.3025 - accuracy: 0.8891 - val_loss: 0.2763 - val_accuracy: 0.8926 - 45ms/epoch - 1ms/step
Epoch 3/100
36/36 - 0s - loss: 0.2913 - accuracy: 0.8908 - val_loss: 0.2733 - val_accuracy: 0.8993 - 47ms/epoch - 1ms/step
Epoch 4/100
36/36 - 0s - loss: 0.2801 - accuracy: 0.8947 - val_loss: 0.2654 - val_accuracy: 0.8971 - 47ms/epoch - 1ms/step
Epoch 5/100
36/36 - 0s - loss: 0.2716 - accuracy: 0.8980 - val_loss: 0.2649 - val_accuracy: 0.8993 - 47ms/epoch - 1ms/step
Epoch 6/100
36/36 - 0s - loss: 0.2666 - accuracy: 0.8994 - val_loss: 0.2595 - val_accuracy: 0.8993 - 45ms/epoch - 1ms/step
Epoch 7/100
36/36 - 0s - loss: 0.2608 - accuracy: 0.9022 - val_loss: 0.2580 - val_accuracy: 0.9060 - 47ms/epoch - 1ms/step
Epoch 8/100
36/36 - 0s - loss: 0.2574 - accuracy: 0.9036 - val_loss: 0.2541 - val_accuracy: 0.9038 - 46ms/epoch - 1ms/step
Epoch 9/100
36/3

**Testeando el modelo**

In [43]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 769us/step - loss: 0.2531 - accuracy: 0.9062
